# 커피매장 전처리
- 특별시, 광역시 : 시,군으로 나누고 더 나아가 구까지 작성
- 도, 지방도시 : 시,군

- 칼럼명, 순서 : ['브랜드명', '지점명', '시도명', '시군구명', '주소']

In [ ]:
import requests
import json
import pandas as pd
from tqdm import trange
import time

import folium
from bs4 import BeautifulSoup as bs

## 투썸플레이스

In [157]:
df = pd.read_csv("twosome.csv")

In [158]:
df.head()

,매장명,지역,브랜드명
0,철원동송점,강원도 철원군 동송읍 금학로 242 (새마을금고) 금학로 242,투썸플레이스
1,수지동천역점,"경기도 용인시 수지구 신수로 767 (동천동,분당수지U-TOWER) 에이동 1층 에...",투썸플레이스
2,광주오치점,광주광역시 북구 설죽로 328 (오치동),투썸플레이스
3,철원주상절리점,강원 철원군 갈말읍 군탄리 1226-1,투썸플레이스
4,별내별가람역점,"경기도 남양주시 덕송1로55번길 22 (별내동) 1층,2층",투썸플레이스


In [159]:
# 지역에서 앞뒤 공백 제거
df['지역'] = df['지역'].str.strip()

In [160]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시도명'] = df['지역'].str.split(' ',n=2).str[0]

In [161]:
df['시도명'].unique()

array(['강원도', '경기도', '광주광역시', '강원', '서울특별시', '인천', '경기', '경북', '충청북도',
       '대전', '충북', '전라북도', '대구광역시', '제주', '서울', '경상남도', '인천광역시', '광주',
       '경상북도', '부산', '부산광역시', '전라남도', '충남', '제주특별자치도', '대전광역시', '전북',
       '서울시', '대구', '전남', '울산', '경남', '충청남도', '세종특별자치시', '울산광역시', '세종',
       '고양시', '세종시', '제주도', '부산시'], dtype=object)

In [162]:
df.loc[df['시도명'] == '강원', '시도명'] = '강원도'
df.loc[df['시도명'] == '인천', '시도명'] = '인천광역시'
df.loc[df['시도명'] == '경기', '시도명'] = '경기도'
df.loc[df['시도명'] == '경북', '시도명'] = '경상북도'
df.loc[df['시도명'] == '대전', '시도명'] = '대전광역시'
df.loc[df['시도명'] == '충북', '시도명'] = '충청북도'
df.loc[df['시도명'] == '제주', '시도명'] = '제주특별자치도'
df.loc[df['시도명'] == '서울', '시도명'] = '서울특별시'
df.loc[df['시도명'] == '광주', '시도명'] = '광주광역시'
df.loc[df['시도명'] == '부산', '시도명'] = '부산광역시'
df.loc[df['시도명'] == '충남', '시도명'] = '충청남도'
df.loc[df['시도명'] == '전북', '시도명'] = '전라북도'
df.loc[df['시도명'] == '서울시', '시도명'] = '서울특별시'
df.loc[df['시도명'] == '대구', '시도명'] = '대구광역시'
df.loc[df['시도명'] == '전남', '시도명'] = '전라남도'
df.loc[df['시도명'] == '울산', '시도명'] = '울산광역시'
df.loc[df['시도명'] == '경남', '시도명'] = '경상남도'
df.loc[df['시도명'] == '세종', '시도명'] = '세종특별자치시'
df.loc[df['시도명'] == '세종시', '시도명'] = '세종특별자치시'
df.loc[df['시도명'] == '제주도', '시도명'] = '제주특별자치도'
df.loc[df['시도명'] == '부산시', '시도명'] = '부산광역시'
df.loc[df['시도명'] == '고양시', '시도명'] = '경기도'

In [163]:
df['시도명'].unique()

array(['강원도', '경기도', '광주광역시', '서울특별시', '인천광역시', '경상북도', '충청북도', '대전광역시',
       '전라북도', '대구광역시', '제주특별자치도', '경상남도', '부산광역시', '전라남도', '충청남도',
       '울산광역시', '세종특별자치시'], dtype=object)

In [164]:
df

,매장명,지역,브랜드명,시도명
0,철원동송점,강원도 철원군 동송읍 금학로 242 (새마을금고) 금학로 242,투썸플레이스,강원도
1,수지동천역점,"경기도 용인시 수지구 신수로 767 (동천동,분당수지U-TOWER) 에이동 1층 에...",투썸플레이스,경기도
2,광주오치점,광주광역시 북구 설죽로 328 (오치동),투썸플레이스,광주광역시
3,철원주상절리점,강원 철원군 갈말읍 군탄리 1226-1,투썸플레이스,강원도
4,별내별가람역점,"경기도 남양주시 덕송1로55번길 22 (별내동) 1층,2층",투썸플레이스,경기도
...,...,...,...,...
1557,광주CGV점,"광주광역시 서구 무진대로 904 2층 (광천동, 유스퀘어광천터미널)",투썸플레이스,광주광역시
1558,CJENM커머스부문점,서울특별시 서초구 과천대로 870-13 CJENM커머스부문점),투썸플레이스,서울특별시
1559,서울대점,"서울특별시 관악구 관악로 1 교수연구동 501동 (신림동, 서울대학교)",투썸플레이스,서울특별시
1560,여의도점,"서울특별시 영등포구 의사당대로 108 1층 2층 (여의도동, 아일렉스상가)",투썸플레이스,서울특별시


In [165]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시군구명'] = df['지역'].str.split(' ',n=2).str[1]

In [166]:
df

,매장명,지역,브랜드명,시도명,시군구명
0,철원동송점,강원도 철원군 동송읍 금학로 242 (새마을금고) 금학로 242,투썸플레이스,강원도,철원군
1,수지동천역점,"경기도 용인시 수지구 신수로 767 (동천동,분당수지U-TOWER) 에이동 1층 에...",투썸플레이스,경기도,용인시
2,광주오치점,광주광역시 북구 설죽로 328 (오치동),투썸플레이스,광주광역시,북구
3,철원주상절리점,강원 철원군 갈말읍 군탄리 1226-1,투썸플레이스,강원도,철원군
4,별내별가람역점,"경기도 남양주시 덕송1로55번길 22 (별내동) 1층,2층",투썸플레이스,경기도,남양주시
...,...,...,...,...,...
1557,광주CGV점,"광주광역시 서구 무진대로 904 2층 (광천동, 유스퀘어광천터미널)",투썸플레이스,광주광역시,서구
1558,CJENM커머스부문점,서울특별시 서초구 과천대로 870-13 CJENM커머스부문점),투썸플레이스,서울특별시,서초구
1559,서울대점,"서울특별시 관악구 관악로 1 교수연구동 501동 (신림동, 서울대학교)",투썸플레이스,서울특별시,관악구
1560,여의도점,"서울특별시 영등포구 의사당대로 108 1층 2층 (여의도동, 아일렉스상가)",투썸플레이스,서울특별시,영등포구


In [167]:
df['시군구명'].unique()

array(['철원군', '용인시', '북구', '남양주시', '용산구', '부천시', '서구', '가평군', '하남시',
       '성남시', '고양시', '안동시', '김포시', '청주시', '대덕구', '충주시', '익산시', '수원시',
       '달성군', '서귀포시', '종로구', '화성시', '중구', '평택시', '김해시', '연수구', '광산구',
       '유성구', '영등포구', '거제시', '포항시', '남구', '제천시', '송파구', '미추홀구', '과천시',
       '연제구', '경산시', '광진구', '문경시', '강진군', '삼척시', '서초구', '아산시', '동래구',
       '안산시', '제주시', '금천구', '진천군', '진주시', '달서구', '강북구', '청도군', '군포시',
       '동구', '강동구', '속초시', '전주시', '성동구', '담양군', '고흥군', '시흥시', '군산시',
       '수성구', '강서구', '남동구', '부평구', '여주시', '서대문구', '순천시', '공주시', '여수시',
       '마포구', '춘천시', '군위군', '고성군', '성북구', '나주시', '홍천군', '포천시', '창원시',
       '이천시', '음성군', '의정부시', '광명시', '구미시', '수영구', '강남구', '연천군', '양천구',
       '원주시', '완주군', '광양시', '계양구', '당진시', '해밀3로', '파주시', '안양시', '오산시',
       '해운대구', '양주시', '노원구', '은평구', '천안시', '구리시', '구로구', '기장군', '보령시',
       '영주시', '사상구', '부여군', '통영시', '예산군', '계룡시', '금정구', '의왕시', '무안군',
       '부산진구', '한누리대로', '동대문구', '보듬3로', '양평군', '중랑구', '강화군', '영월군', '서산시',
       '칠곡군'

In [171]:
df.loc[df['시군구명'] == '해밀3로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '보듬3로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '한누리대로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '아름동', '시군구명'] = '세종특별자치시'

df.loc[df['시군구명'] == '조치원읍', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '고운동', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '마음안1로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '도움8로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '다정중앙로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '애월읍', '시군구명'] = '제주특별자치도'

In [172]:
df['시군구명'].unique()

array(['철원군', '용인시', '북구', '남양주시', '용산구', '부천시', '서구', '가평군', '하남시',
       '성남시', '고양시', '안동시', '김포시', '청주시', '대덕구', '충주시', '익산시', '수원시',
       '달성군', '서귀포시', '종로구', '화성시', '중구', '평택시', '김해시', '연수구', '광산구',
       '유성구', '영등포구', '거제시', '포항시', '남구', '제천시', '송파구', '미추홀구', '과천시',
       '연제구', '경산시', '광진구', '문경시', '강진군', '삼척시', '서초구', '아산시', '동래구',
       '안산시', '제주시', '금천구', '진천군', '진주시', '달서구', '강북구', '청도군', '군포시',
       '동구', '강동구', '속초시', '전주시', '성동구', '담양군', '고흥군', '시흥시', '군산시',
       '수성구', '강서구', '남동구', '부평구', '여주시', '서대문구', '순천시', '공주시', '여수시',
       '마포구', '춘천시', '군위군', '고성군', '성북구', '나주시', '홍천군', '포천시', '창원시',
       '이천시', '음성군', '의정부시', '광명시', '구미시', '수영구', '강남구', '연천군', '양천구',
       '원주시', '완주군', '광양시', '계양구', '당진시', '세종특별자치시', '파주시', '안양시', '오산시',
       '해운대구', '양주시', '노원구', '은평구', '천안시', '구리시', '구로구', '기장군', '보령시',
       '영주시', '사상구', '부여군', '통영시', '예산군', '계룡시', '금정구', '의왕시', '무안군',
       '부산진구', '동대문구', '양평군', '중랑구', '강화군', '영월군', '서산시', '칠곡군', '평창군',
       '도봉구'

In [173]:
df

,매장명,지역,브랜드명,시도명,시군구명
0,철원동송점,강원도 철원군 동송읍 금학로 242 (새마을금고) 금학로 242,투썸플레이스,강원도,철원군
1,수지동천역점,"경기도 용인시 수지구 신수로 767 (동천동,분당수지U-TOWER) 에이동 1층 에...",투썸플레이스,경기도,용인시
2,광주오치점,광주광역시 북구 설죽로 328 (오치동),투썸플레이스,광주광역시,북구
3,철원주상절리점,강원 철원군 갈말읍 군탄리 1226-1,투썸플레이스,강원도,철원군
4,별내별가람역점,"경기도 남양주시 덕송1로55번길 22 (별내동) 1층,2층",투썸플레이스,경기도,남양주시
...,...,...,...,...,...
1557,광주CGV점,"광주광역시 서구 무진대로 904 2층 (광천동, 유스퀘어광천터미널)",투썸플레이스,광주광역시,서구
1558,CJENM커머스부문점,서울특별시 서초구 과천대로 870-13 CJENM커머스부문점),투썸플레이스,서울특별시,서초구
1559,서울대점,"서울특별시 관악구 관악로 1 교수연구동 501동 (신림동, 서울대학교)",투썸플레이스,서울특별시,관악구
1560,여의도점,"서울특별시 영등포구 의사당대로 108 1층 2층 (여의도동, 아일렉스상가)",투썸플레이스,서울특별시,영등포구


In [174]:
df.columns = [['지점명','주소','브랜드명','시도명','시군구명']]

In [175]:
df = df[['브랜드명', '지점명', '시도명', '시군구명', '주소']]

In [176]:
file_name = "twosome_cleaning.csv"
df.to_csv(file_name,index = False ,  encoding="utf-8-sig")

## 엔제리너스

In [57]:
df = pd.read_csv("angelinus.csv")

In [58]:
df.head()

,지역,매장명,브랜드명
0,충청남도 태안군 태안읍 702-1,충남태안점,엔제리너스
1,충청남도 서산시 안견로 191,서산터미널점,엔제리너스
2,"경기도 시흥시 거북섬5길4 114,115,206,207호",시화거북섬,엔제리너스
3,인천광역시 연수구 컨벤시아대로 177 롯데마트,롯데마트송도점,엔제리너스
4,충청남도 홍성군 홍학로 87 엘가프라자,홍성내포엘가점,엔제리너스


In [59]:
# 지역에서 앞뒤 공백 제거
df['지역'] = df['지역'].str.strip()

In [60]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시도명'] = df['지역'].str.split(' ',n=2).str[0]

In [61]:
df['시도명'].unique()

array(['충청남도', '경기도', '인천광역시', '서울특별시', '세종특별자치시', '충청북도', '전라북도',
       '대전광역시', '강원도', '전라남도', '경상북도', '충청', '광주광역시', '대구광역시', '경상남도',
       '울산광역시', '부산광역시', '제주특별자치도'], dtype=object)

In [62]:
df.loc[df['시도명'] == '충청', '시도명'] = '충청북도'

In [63]:
df['시도명'].unique()

array(['충청남도', '경기도', '인천광역시', '서울특별시', '세종특별자치시', '충청북도', '전라북도',
       '대전광역시', '강원도', '전라남도', '경상북도', '광주광역시', '대구광역시', '경상남도', '울산광역시',
       '부산광역시', '제주특별자치도'], dtype=object)

In [64]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시군구명'] = df['지역'].str.split(' ',n=2).str[1]

In [65]:
df['시군구명'].unique()

array(['태안군', '서산시', '시흥시', '연수구', '홍성군', '중구', '화성시', '미추홀구', '서구',
       '계양구', '부천시', '수원시', '보령시', '아산시', '안양시', '광명시', '김포시', '강서구',
       '금천구', '평택시', '관악구', '구로구', '오산시', '영등포구', '고양시', '용인시', '동작구',
       '마포구', '천안시', '서대문구', '용산구', '서초구', '은평구', '강남구', '종로구', '성남시',
       '광진구', '송파구', '동대문구', '강북구', '중랑구', '노원구', '강동구', '구리시', '하남시',
       '조치원읍', '의정부시', '청주시', '', '군산시', '동두천시', '유성구', '익산시', '여주군',
       '동구', '완주군', '옥천군', '전주시', '충주시', '괴산군', '원주시', '춘천시', '영광군읍',
       '문경시', '단양군', '김천시', '광산구', '남원시', '구미시', '북구', '곡성군', '남구', '인제군',
       '의성군', '칠곡군', '안동시', '목포시', '무안군', '달성군', '달서구', '속초시', '순천시',
       '수성구', '진주시', '의령군', '광양시', '경산시', '함안군', '여수시', '사천시', '울진군',
       '경주시', '창원시', '울진군후포면', '포항시', '김해시', '울주군', '통영시', '울주근', '사상구',
       '거제시', '금정구', '사하구', '동래구', '부산진구', '해운대구', '수영구', '기장군', '영도구',
       '제주시', '서귀포시'], dtype=object)

In [66]:
df.loc[df['시군구명'] == '조치원읍', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '영광군읍', '시군구명'] = '영광군'
df.loc[df['시군구명'] == '울진군후포면', '시군구명'] = '울진군'
df.loc[df['시군구명'] == '울주근', '시군구명'] = '울주군'

In [67]:
df['시군구명'].unique()

array(['태안군', '서산시', '시흥시', '연수구', '홍성군', '중구', '화성시', '미추홀구', '서구',
       '계양구', '부천시', '수원시', '보령시', '아산시', '안양시', '광명시', '김포시', '강서구',
       '금천구', '평택시', '관악구', '구로구', '오산시', '영등포구', '고양시', '용인시', '동작구',
       '마포구', '천안시', '서대문구', '용산구', '서초구', '은평구', '강남구', '종로구', '성남시',
       '광진구', '송파구', '동대문구', '강북구', '중랑구', '노원구', '강동구', '구리시', '하남시',
       '세종특별자치시', '의정부시', '청주시', '군산시', '동두천시', '유성구', '익산시', '여주군', '동구',
       '완주군', '옥천군', '전주시', '충주시', '괴산군', '원주시', '춘천시', '영광군', '문경시',
       '단양군', '김천시', '광산구', '남원시', '구미시', '북구', '곡성군', '남구', '인제군', '의성군',
       '칠곡군', '안동시', '목포시', '무안군', '달성군', '달서구', '속초시', '순천시', '수성구',
       '진주시', '의령군', '광양시', '경산시', '함안군', '여수시', '사천시', '울진군', '경주시',
       '창원시', '포항시', '김해시', '울주군', '통영시', '사상구', '거제시', '금정구', '사하구',
       '동래구', '부산진구', '해운대구', '수영구', '기장군', '영도구', '제주시', '서귀포시'],
      dtype=object)

In [69]:
df

,지역,매장명,브랜드명,시도명,시군구명
0,충청남도 태안군 태안읍 702-1,충남태안점,엔제리너스,충청남도,태안군
1,충청남도 서산시 안견로 191,서산터미널점,엔제리너스,충청남도,서산시
2,"경기도 시흥시 거북섬5길4 114,115,206,207호",시화거북섬,엔제리너스,경기도,시흥시
3,인천광역시 연수구 컨벤시아대로 177 롯데마트,롯데마트송도점,엔제리너스,인천광역시,연수구
4,충청남도 홍성군 홍학로 87 엘가프라자,홍성내포엘가점,엔제리너스,충청남도,홍성군
...,...,...,...,...,...
301,제주특별자치도 제주시 공항로,주공항1층점,엔제리너스,제주특별자치도,제주시
302,제주특별자치도 제주시 서해안로 24,주이호테우해변로점,엔제리너스,제주특별자치도,제주시
303,제주특별자치도 제주시 연북로 514,주연북로점,엔제리너스,제주특별자치도,제주시
304,제주특별자치도 제주시 수목원길 65-6,한라수목원점,엔제리너스,제주특별자치도,제주시


In [70]:
df.columns = [['주소','지점명','브랜드명','시도명','시군구명']]

In [71]:
df = df[['브랜드명', '지점명', '시도명', '시군구명', '주소']]

In [72]:
file_name = "angelinus_cleaning.csv"
df.to_csv(file_name,index = False ,  encoding="utf-8-sig")

In [68]:
# 시도명이 "도"인데 시군구명이 "시"나 "군"이 아닌 경우
df[(df['시도명'].str[-1]=='도') & ~(df['시군구명'].str[-1].isin(['시','군']))]

# 시도명이 "시"인데 시군구명이 "구"가 아닌 경우
df[((df['시도명'].str[-1]=='시') & ~(df['시군구명'].str[-1]=='구'))]

,지역,매장명,브랜드명,시도명,시군구명
118,세종특별자치시 조치원읍 행복11길 3,조치원점,엔제리너스,세종특별자치시,세종특별자치시
121,세종특별자치시 한누리대로 2147,세종시청점,엔제리너스,세종특별자치시,세종특별자치시
199,대구광역시 달성군 광주대구고속도로 170,논공휴게소상,엔제리너스,대구광역시,달성군
200,대구광역시 달성군 논공읍 광주대고고속도로 169,논공휴게소하,엔제리너스,대구광역시,달성군
246,울산광역시 울주군 언양읍 반곡리 966-15 (언동3길 20-48 엔제리너스),언양휴게소점,엔제리너스,울산광역시,울주군
248,울산광역시 울주군 봉화로 10,언양삼남점,엔제리너스,울산광역시,울주군
253,울산광역시 울주근 범서읍 울밀로 2848-3,울산굴화DI,엔제리너스,울산광역시,울주군
290,부산광역시 기장군 차성동로 110 엔제리너스 기장점,부산기장점,엔제리너스,부산광역시,기장군
296,부산광역시 기장군 기장읍 당사리 64 1F,롯데아울렛동부산1층점,엔제리너스,부산광역시,기장군
297,부산광역시 기장군 기장읍 당사리 64 3F,롯데아울렛동부산3층점,엔제리너스,부산광역시,기장군


## 폴바셋

In [132]:
df = pd.read_csv("paulbassett_store.csv")

In [133]:
df.head()

,브랜드명,shopName,address
0,폴바셋,용인 에버랜드점,경기도 용인시 처인구 포곡읍 에버랜드로 199
1,폴바셋,캐리비안베이점(시즌 종료),경기도 용인시 처인구 포곡읍 에버랜드로 199
2,폴바셋,AK플라자 분당점,경기도 성남시 분당구 황새울로360번길 42AK플라자 분당점 지하 1층
3,폴바셋,용인시청 DT점,경기도 용인시 처인구 삼가동 198-15
4,폴바셋,롯데백화점 분당점,경기도 성남시 분당구 황새울로 200번길 45 롯데백화점 분당점 1층


In [134]:
df = df.rename(columns={'address':'지역'})

In [135]:
df.head()

,브랜드명,shopName,지역
0,폴바셋,용인 에버랜드점,경기도 용인시 처인구 포곡읍 에버랜드로 199
1,폴바셋,캐리비안베이점(시즌 종료),경기도 용인시 처인구 포곡읍 에버랜드로 199
2,폴바셋,AK플라자 분당점,경기도 성남시 분당구 황새울로360번길 42AK플라자 분당점 지하 1층
3,폴바셋,용인시청 DT점,경기도 용인시 처인구 삼가동 198-15
4,폴바셋,롯데백화점 분당점,경기도 성남시 분당구 황새울로 200번길 45 롯데백화점 분당점 1층


In [136]:
# 지역에서 앞뒤 공백 제거
df['지역'] = df['지역'].str.strip()

In [137]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시도명'] = df['지역'].str.split(' ',n=2).str[0]

In [138]:
df['시도명'].unique()

array(['경기도', '서울시', '서울특별시', '서울', '경기', '광명시', '안성시', '인천시', '인천광역시',
       '인천', '충청북도', '대전시', '대전광역시', '전주시', '대구광역시', '대구', '대구시', '경상북도',
       '광주광역시', '창원시', '경남', '울산광역시', '경상남도', '부산시', '부산광역시', '제주특별자치도',
       '제주시'], dtype=object)

In [139]:
df.loc[df['시도명'] == '서울시', '시도명'] = '서울특별시'
df.loc[df['시도명'] == '서울', '시도명'] = '서울특별시'
df.loc[df['시도명'] == '광명시', '시도명'] = '경기도'
df.loc[df['시도명'] == '안성시', '시도명'] = '경기도'
df.loc[df['시도명'] == '인천시', '시도명'] = '인천특별시'
df.loc[df['시도명'] == '대전시', '시도명'] = '대전광역시'
df.loc[df['시도명'] == '전주시', '시도명'] = '전라북도'
df.loc[df['시도명'] == '대구', '시도명'] = '대구광역시'
df.loc[df['시도명'] == '대구시', '시도명'] = '대구광역시'
df.loc[df['시도명'] == '창원시', '시도명'] = '경상남도'
df.loc[df['시도명'] == '경남', '시도명'] = '경상남도'
df.loc[df['시도명'] == '부산시', '시도명'] = '부산광역시'
df.loc[df['시도명'] == '제주시', '시도명'] = '제주특별자치도'

In [140]:
df['시도명'].unique()

array(['경기도', '서울특별시', '경기', '인천특별시', '인천광역시', '인천', '충청북도', '대전광역시',
       '전라북도', '대구광역시', '경상북도', '광주광역시', '경상남도', '울산광역시', '부산광역시',
       '제주특별자치도'], dtype=object)

In [141]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시군구명'] = df['지역'].str.split(' ',n=2).str[1]

In [142]:
df['시군구명'].unique()

array(['용인시', '성남시', '하남시', '강남구', '수원시', '강동구', '송파구', '화성시', '여주시',
       '서초구', '오산시', '남양주시', '안양시', '용산구', '동대문구', '중구', '관악구', '종로구',
       '광명시', '영등포구', '구로구', '노원구', '마포구', '오리로', '공도읍', '강서구', '은평구',
       '의정부시', '고양시', '시흥시', '부천시', '김포시', '연수구', '파주시', '청주시', '유성구',
       '서구', '덕진구', '완산구', '북구', '동구', '수성구', '경산시', '광산구', '마산합포구',
       '창원시', '남구', '김해시', '부산진구', '해운대구', '기장군', '제주시', '아라이동'],
      dtype=object)

In [143]:
df.loc[df['시군구명'] == '오리로', '시군구명'] = '안양시'
df.loc[df['시군구명'] == '공도읍', '시군구명'] = '안성시'
df.loc[df['시군구명'] == '덕진구', '시군구명'] = '전주시'
df.loc[df['시군구명'] == '완산구', '시군구명'] = '전주시'
df.loc[df['시군구명'] == '마산합포구', '시군구명'] = '창원시'
df.loc[df['시군구명'] == '아라이동', '시군구명'] = '제주시'

In [144]:
df.head()

,브랜드명,shopName,지역,시도명,시군구명
0,폴바셋,용인 에버랜드점,경기도 용인시 처인구 포곡읍 에버랜드로 199,경기도,용인시
1,폴바셋,캐리비안베이점(시즌 종료),경기도 용인시 처인구 포곡읍 에버랜드로 199,경기도,용인시
2,폴바셋,AK플라자 분당점,경기도 성남시 분당구 황새울로360번길 42AK플라자 분당점 지하 1층,경기도,성남시
3,폴바셋,용인시청 DT점,경기도 용인시 처인구 삼가동 198-15,경기도,용인시
4,폴바셋,롯데백화점 분당점,경기도 성남시 분당구 황새울로 200번길 45 롯데백화점 분당점 1층,경기도,성남시


In [145]:
df.columns = [['브랜드명','지점명','주소','시도명','시군구명']]

In [146]:
df = df[['브랜드명', '지점명', '시도명', '시군구명', '주소']]

In [147]:
file_name = "paulbassett_cleaning.csv"
df.to_csv(file_name,index = False ,  encoding="utf-8-sig")

## 폴바셋 추가


In [2]:
import os
os.getcwd()
os.chdir('C:/Users/pmj03/likelion') #절대 경로 설정
df = pd.read_csv("./MID/coffee/cleaning\\paulbassett_cleaning.csv")

In [3]:
df['시도명'].unique()

array(['경기도', '서울특별시', '경기', '인천특별시', '인천광역시', '인천', '충청북도', '대전광역시',
       '전라북도', '대구광역시', '경상북도', '광주광역시', '경상남도', '울산광역시', '부산광역시',
       '제주특별자치도'], dtype=object)

In [5]:
df.loc[df['시도명'] == '경기', '시도명'] = '경기도'
df.loc[df['시도명'] == '인천특별시', '시도명'] = '인천광역시'
df.loc[df['시도명'] == '인천', '시도명'] = '인천광역시'

In [6]:
df['시도명'].unique()

array(['경기도', '서울특별시', '인천광역시', '충청북도', '대전광역시', '전라북도', '대구광역시', '경상북도',
       '광주광역시', '경상남도', '울산광역시', '부산광역시', '제주특별자치도'], dtype=object)

In [7]:
df['시군구명'].unique()

array(['용인시', '성남시', '하남시', '강남구', '수원시', '강동구', '송파구', '화성시', '여주시',
       '서초구', '오산시', '남양주시', '안양시', '용산구', '동대문구', '중구', '관악구', '종로구',
       '광명시', '영등포구', '구로구', '노원구', '마포구', '안성시', '강서구', '은평구', '의정부시',
       '고양시', '시흥시', '부천시', '김포시', '연수구', '파주시', '청주시', '유성구', '서구',
       '전주시', '북구', '동구', '수성구', '경산시', '광산구', '창원시', '남구', '김해시', '부산진구',
       '해운대구', '기장군', '제주시'], dtype=object)

In [11]:
file_name = "./MID/coffee/cleaning/paulbassett_cleaning_last.csv"
df.to_csv(file_name,index = False ,  encoding="utf-8-sig")

## 컴포즈커피

In [35]:
df = pd.read_csv("composecoffee.csv")

In [36]:
df.head()

,지역,매장명,브랜드명
0,충청남도 서산시 예천동 1261-3 컴포즈커피(서산호수공원점),서산호수공원점,컴포즈커피
1,충청남도 서산시 석림1로 68 컴포즈커피(서산석림점),서산석림점,컴포즈커피
2,"인천광역시 중구 흰바위로 236 1층119호(운남동,영종메가타워)",인천영종메가타워점,컴포즈커피
3,충청남도 당진시 송악읍 신복운로3길 23-1 1층,당진송악점,컴포즈커피
4,인천광역시 연수구 하모니로 271 제1동 102호,송도인천대점,컴포즈커피


In [37]:
# 지역에서 앞뒤 공백 제거
df['지역'] = df['지역'].str.strip()

In [38]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시도명'] = df['지역'].str.split(' ',n=2).str[0]

In [39]:
df['시도명'].unique()

array(['충청남도', '인천광역시', '경기도', '서울특별시', '세종특별자치시', '전라북도', '충청북도',
       '대전광역시', '강원도', '전라남도', '경상북도', '광주광역시', '경상남도', '대구광역시', '창원시',
       '울산광역시', '부산광역시', '제주특별자치도'], dtype=object)

In [41]:
df.loc[df['시도명'] == '창원시', '시도명'] = '경상남도'

In [42]:
df['시도명'].unique()

array(['충청남도', '인천광역시', '경기도', '서울특별시', '세종특별자치시', '전라북도', '충청북도',
       '대전광역시', '강원도', '전라남도', '경상북도', '광주광역시', '경상남도', '대구광역시', '울산광역시',
       '부산광역시', '제주특별자치도'], dtype=object)

In [43]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시군구명'] = df['지역'].str.split(' ',n=2).str[1]

In [44]:
df['시군구명'].unique()

array(['서산시', '중구', '당진시', '연수구', '예산군', '홍성군', '화성시', '시흥시', '안산시',
       '미추홀구', '남동구', '동구', '서구', '부평구', '평택시', '김포시', '부천시', '계양구',
       '아산시', '수원시', '군포시', '구로구', '보령시', '용인시', '광명시', '의왕시', '안양시',
       '강화군', '성남시', '양천구', '금천구', '오산시', '강서구', '관악구', '동작구', '영등포구',
       '고양시', '천안시', '과천시', '동작루', '파주시', '마포구', '서초구', '남양주시', '은평구',
       '서대문구', '용산구', '종로구', '강남구', '성동구', '광주시', '송파구', '성북구', '광진구',
       '안성시', '동대문구', '강북구', '도봉구', '중랑구', '하남시', '강동구', '공주시', '노원구',
       '구리시', '의정부시', '이천시', '보듬3로', '새롬중앙로', '군산시', '한누리대로', '양주시',
       '청주시', '논산시', '유성구', '포천시', '음성군', '익산시', '연천군', '여주시', '옥천군',
       '완주군', '전주시', '가평군', '원주시', '충주시', '철원군', '정읍시', '화천군', '춘천시',
       '홍천군', '횡성군', '고창군', '제천시', '영광군', '장성군', '상주시', '북구', '광산구',
       '남원시', '영월군', '남구', '예천군', '거창군', '구미시', '나주시', '무안군', '인제군',
       '화순군', '칠곡군', '영주시', '성주군', '산청군', '안동시', '목포시', '달성군', '양양군',
       '달서구', '속초시', '순천시', '보성군', '수성구', '강릉시', '광양시', '장흥군', '진주시',
       '해남군', '창녕군', 

In [46]:
df.loc[df['시군구명'] == '동작루', '시군구명'] = '동작구'
df.loc[df['시군구명'] == '보듬3로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '새롬중앙로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '한누리대로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '마산합포구', '시군구명'] = '창원시'

In [49]:
df.loc[df['시군구명'] == '해운대구광역시', '시군구명'] = '해운대구'

In [47]:
df['시군구명'].unique()

array(['서산시', '중구', '당진시', '연수구', '예산군', '홍성군', '화성시', '시흥시', '안산시',
       '미추홀구', '남동구', '동구', '서구', '부평구', '평택시', '김포시', '부천시', '계양구',
       '아산시', '수원시', '군포시', '구로구', '보령시', '용인시', '광명시', '의왕시', '안양시',
       '강화군', '성남시', '양천구', '금천구', '오산시', '강서구', '관악구', '동작구', '영등포구',
       '고양시', '천안시', '과천시', '파주시', '마포구', '서초구', '남양주시', '은평구', '서대문구',
       '용산구', '종로구', '강남구', '성동구', '광주시', '송파구', '성북구', '광진구', '안성시',
       '동대문구', '강북구', '도봉구', '중랑구', '하남시', '강동구', '공주시', '노원구', '구리시',
       '의정부시', '이천시', '세종특별자치시', '군산시', '양주시', '청주시', '논산시', '유성구', '포천시',
       '음성군', '익산시', '연천군', '여주시', '옥천군', '완주군', '전주시', '가평군', '원주시',
       '충주시', '철원군', '정읍시', '화천군', '춘천시', '홍천군', '횡성군', '고창군', '제천시',
       '영광군', '장성군', '상주시', '북구', '광산구', '남원시', '영월군', '남구', '예천군', '거창군',
       '구미시', '나주시', '무안군', '인제군', '화순군', '칠곡군', '영주시', '성주군', '산청군',
       '안동시', '목포시', '달성군', '양양군', '달서구', '속초시', '순천시', '보성군', '수성구',
       '강릉시', '광양시', '장흥군', '진주시', '해남군', '창녕군', '경산시', '동해시', '사천시',
       '

In [55]:
df

,브랜드명,지점명,시도명,시군구명,주소
0,컴포즈커피,서산호수공원점,충청남도,서산시,충청남도 서산시 예천동 1261-3 컴포즈커피(서산호수공원점)
1,컴포즈커피,서산석림점,충청남도,서산시,충청남도 서산시 석림1로 68 컴포즈커피(서산석림점)
2,컴포즈커피,인천영종메가타워점,인천광역시,중구,"인천광역시 중구 흰바위로 236 1층119호(운남동,영종메가타워)"
3,컴포즈커피,당진송악점,충청남도,당진시,충청남도 당진시 송악읍 신복운로3길 23-1 1층
4,컴포즈커피,송도인천대점,인천광역시,연수구,인천광역시 연수구 하모니로 271 제1동 102호
...,...,...,...,...,...
1861,컴포즈커피,서귀포하효점,제주특별자치도,서귀포시,제주특별자치도 서귀포시 효돈순환로 246 컴포즈커피
1862,컴포즈커피,서귀포성산점,제주특별자치도,서귀포시,제주특별자치도 서귀포시 성산읍 일주동로 4229 컴포즈커피(서귀포성산점)
1863,컴포즈커피,서귀포올레시장점,제주특별자치도,서귀포시,제주특별자치도 서귀포시 중정로 66-1 컴포즈커피(서귀포올레시장점)
1864,컴포즈커피,서귀포대정점,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 신영로 102 컴포즈커피 서귀포대정점


In [52]:
df.columns = [['주소','지점명','브랜드명','시도명','시군구명']]

In [53]:
df = df[['브랜드명', '지점명', '시도명', '시군구명', '주소']]

In [56]:
file_name = "compose_cleaning.csv"
df.to_csv(file_name,index = False ,  encoding="utf-8-sig")

In [51]:
# 시도명이 "도"인데 시군구명이 "시"나 "군"이 아닌 경우
df[(df['시도명'].str[-1]=='도') & ~(df['시군구명'].str[-1].isin(['시','군']))]

# 시도명이 "시"인데 시군구명이 "구"가 아닌 경우
df[((df['시도명'].str[-1]=='시') & ~(df['시군구명'].str[-1]=='구'))]

,지역,매장명,브랜드명,시도명,시군구명
342,인천광역시 강화군 강화읍 강화대로 423-1 컴포즈커피(인천강화점),인천강화점,컴포즈커피,인천광역시,강화군
948,"세종특별자치시 보듬3로 100, 109 컴포즈커피 세종아름점",세종아름점,컴포즈커피,세종특별자치시,세종특별자치시
958,"세종특별자치시 새롬중앙로 47 1층 108호(새롬동, 세움빌딩)",세종새롬점,컴포즈커피,세종특별자치시,세종특별자치시
968,세종특별자치시 한누리대로 2143 113호,세종시청점,컴포즈커피,세종특별자치시,세종특별자치시
1237,대구광역시 달성군 다사읍 궁산2길 12 301동 1층 102호,대구서재점,컴포즈커피,대구광역시,달성군
1261,대구광역시 달성군 유가읍 테크노상업로4길 7-4 컴포즈커피(현풍테크노폴리스점),현풍테크노폴리스점,컴포즈커피,대구광역시,달성군
1471,울산광역시 울주군 언양읍 헌양길 62 컴포즈커피(울산언양점),울산언양점,컴포즈커피,울산광역시,울주군
1527,울산광역시 울주군 범서읍 천상중앙길 87 컴포즈커피(울산천상점),울산천상점,컴포즈커피,울산광역시,울주군
1529,울산광역시 울주군 범서읍 점촌3길 5 컴포즈커피(울산범서점),울산범서점,컴포즈커피,울산광역시,울주군
1616,부산광역시 기장군 정관읍 정관로 385 컴포즈커피 정관1호점,정관1호점,컴포즈커피,부산광역시,기장군


## 메가커피

In [73]:
df = pd.read_csv("megacoffee.csv")

In [74]:
df.head()

,지역,매장명,브랜드명
0,충청남도 태안군 태안읍 동문2길 25,태안점,메가커피
1,충청남도 서산시 성연면 성연3로 45 테크노프라자101호,서산테크노밸리점,메가커피
2,충청남도 서산시 호수공원3로 56 (예천동),서산호수공원점,메가커피
3,충청남도 서산시 예천1로 24,서산예천점,메가커피
4,충청남도 서산시 고운로 275-5,서산동문점,메가커피


In [75]:
# 지역에서 앞뒤 공백 제거
df['지역'] = df['지역'].str.strip()

In [76]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시도명'] = df['지역'].str.split(' ',n=2).str[0]

In [77]:
df['시도명'].unique()

array(['충청남도', '경기도', '인천광역시', '서울특별시', '세종특별자치시', '충청북도', '전라북도',
       '대전광역시', '강원도', '전라남도', '경상북도', '광주광역시', '경상남도', '대구광역시', '부산광역시',
       '울산광역시', '제주특별자치도'], dtype=object)

In [78]:
# 띄어쓰기 기준 두번 나눈뒤 시도명 컬럼 생성해주기
df['시군구명'] = df['지역'].str.split(' ',n=2).str[1]

In [79]:
df['시군구명'].unique()

array(['태안군', '서산시', '당진시', '안산시', '중구', '화성시', '시흥시', '연수구', '홍성군',
       '평택시', '미추홀구', '남동구', '서구', '동구', '안산지', '부평구', '예산군', '부천시',
       '계양구', '아산시', '김포시', '수원시', '군포시', '보령시', '구로구', '광명시', '안양시',
       '의왕시', '양천구', '청양군', '금천구', '오산시', '강서구', '영등포구', '관악구', '동작구',
       '용인시', '고양시', '천안시', '과천시', '마포구', '파주시', '안성시', '서초구', '은평구',
       '서대문구', '성남시', '용산구', '강남구', '종로구', '성동구', '광주시', '나성북1로', '성북구',
       '동대문구', '송파구', '광진구', '강북구', '부여군', '하남시', '양주시', '남양주시', '중랑구',
       '강동구', '도봉구', '노원구', '공주시', '구리시', '의정부시', '조치원읍', '이천시', '보듬3로',
       '도움1로', '보듬4로', '다정중앙로', '청주시', '해밀3로', '가름로', '새롬중앙로', '한누리대로',
       '진천군', '소담1로', '군산시', '논산시', '유성구', '동두천시', '포천시', '음성군', '대덕구',
       '증평군', '양평군', '익산시', '연천군', '여주시', '가평군', '완주군', '김제시', '전주시',
       '옥천군', '괴산군', '충주시', '보은군', '원주시', '금산군', '철원군', '정읍시', '춘천시',
       '화천군', '영동군', '홍천군', '고창군', '제천시', '영광군', '장수군', '문경시', '장성군',
       '상주시', '담양군', '순창군', '단양군', '북구', '광산구', '김천시', '인제군', '양구군',
       '남원

In [80]:
df.loc[df['시군구명'] == '나성북1로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '조치원읍', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '보듬3로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '도움1로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '보듬4로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '다정중앙로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '해밀3로', '시군구명'] = '세종특별자치시'

df.loc[df['시군구명'] == '가름로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '새롬중앙로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '한누리대로', '시군구명'] = '세종특별자치시'
df.loc[df['시군구명'] == '소담1로', '시군구명'] = '세종특별자치시'

In [81]:
df['시군구명'].unique()

array(['태안군', '서산시', '당진시', '안산시', '중구', '화성시', '시흥시', '연수구', '홍성군',
       '평택시', '미추홀구', '남동구', '서구', '동구', '안산지', '부평구', '예산군', '부천시',
       '계양구', '아산시', '김포시', '수원시', '군포시', '보령시', '구로구', '광명시', '안양시',
       '의왕시', '양천구', '청양군', '금천구', '오산시', '강서구', '영등포구', '관악구', '동작구',
       '용인시', '고양시', '천안시', '과천시', '마포구', '파주시', '안성시', '서초구', '은평구',
       '서대문구', '성남시', '용산구', '강남구', '종로구', '성동구', '광주시', '세종특별자치시', '성북구',
       '동대문구', '송파구', '광진구', '강북구', '부여군', '하남시', '양주시', '남양주시', '중랑구',
       '강동구', '도봉구', '노원구', '공주시', '구리시', '의정부시', '이천시', '청주시', '진천군',
       '군산시', '논산시', '유성구', '동두천시', '포천시', '음성군', '대덕구', '증평군', '양평군',
       '익산시', '연천군', '여주시', '가평군', '완주군', '김제시', '전주시', '옥천군', '괴산군',
       '충주시', '보은군', '원주시', '금산군', '철원군', '정읍시', '춘천시', '화천군', '영동군',
       '홍천군', '고창군', '제천시', '영광군', '장수군', '문경시', '장성군', '상주시', '담양군',
       '순창군', '단양군', '북구', '광산구', '김천시', '인제군', '양구군', '남원시', '함평군', '남구',
       '거창군', '나주시', '구미시', '무안군', '영주시', '화순군', '봉화군', '목포시', '안동시',
       

In [82]:
df

,지역,매장명,브랜드명,시도명,시군구명
0,충청남도 태안군 태안읍 동문2길 25,태안점,메가커피,충청남도,태안군
1,충청남도 서산시 성연면 성연3로 45 테크노프라자101호,서산테크노밸리점,메가커피,충청남도,서산시
2,충청남도 서산시 호수공원3로 56 (예천동),서산호수공원점,메가커피,충청남도,서산시
3,충청남도 서산시 예천1로 24,서산예천점,메가커피,충청남도,서산시
4,충청남도 서산시 고운로 275-5,서산동문점,메가커피,충청남도,서산시
...,...,...,...,...,...
2201,제주특별자치도 서귀포시 일주동로 8088,서귀포효돈점,메가커피,제주특별자치도,서귀포시
2202,제주특별자치도 서귀포시 동홍동 511-23 1층,제주동홍점,메가커피,제주특별자치도,서귀포시
2203,제주특별자치도 서귀포시 중앙로 56 (서귀동)1층,제주서귀중앙점,메가커피,제주특별자치도,서귀포시
2204,제주특별자치도 서귀포시 태평로 488 (동홍동),제주동홍중앙점,메가커피,제주특별자치도,서귀포시


In [84]:
df.columns = [['주소','지점명','브랜드명','시도명','시군구명']]

In [85]:
df = df[['브랜드명', '지점명', '시도명', '시군구명', '주소']]

In [86]:
file_name = "mega_cleaning.csv"
df.to_csv(file_name,index = False ,  encoding="utf-8-sig")

In [83]:
# 시도명이 "도"인데 시군구명이 "시"나 "군"이 아닌 경우
df[(df['시도명'].str[-1]=='도') & ~(df['시군구명'].str[-1].isin(['시','군']))]

# 시도명이 "시"인데 시군구명이 "구"가 아닌 경우
df[((df['시도명'].str[-1]=='시') & ~(df['시군구명'].str[-1]=='구'))]

,지역,매장명,브랜드명,시도명,시군구명
1118,세종특별자치시 나성북1로 12,세종나성점,메가커피,세종특별자치시,세종특별자치시
1418,세종특별자치시 조치원읍 행복길 25,조치원침산점,메가커피,세종특별자치시,세종특별자치시
1429,세종특별자치시 조치원읍 조치원로 6-1 메가커피,조치원역점,메가커피,세종특별자치시,세종특별자치시
1431,세종특별자치시 보듬3로 92,세종아름점,메가커피,세종특별자치시,세종특별자치시
1439,세종특별자치시 도움1로 106,세종종촌점,메가커피,세종특별자치시,세종특별자치시
1444,"세종특별자치시 보듬4로 9 상가1동 1F-55,56호 (도담동, 카림애비뉴)",세종도담점,메가커피,세종특별자치시,세종특별자치시
1445,세종특별자치시 다정중앙로 52 (다정동) 메가커피,세종다정점,메가커피,세종특별자치시,세종특별자치시
1448,"세종특별자치시 해밀3로 32 지하1층 122호(해밀동, 해밀마을1단지)",세종해밀점,메가커피,세종특별자치시,세종특별자치시
1451,세종특별자치시 가름로 143,세종AK플라자점,메가커피,세종특별자치시,세종특별자치시
1452,"세종특별자치시 새롬중앙로 51 1층 107호 (새롬동, 에스월드빌딩)",세종새롬점,메가커피,세종특별자치시,세종특별자치시
